In [ ]:
%%capture
!pip install alibi

In [ ]:
%%capture
!pip install alibi-detect

In [ ]:
%%capture

!pip install snowflake-connector-python

In [ ]:
%%capture

!pip install snowflake-sqlalchemy

In [ ]:
%%capture

!pip install xgboost

In [27]:
import pandas as pd
import numpy as np 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

import scipy.stats as stats
import matplotlib.pyplot as plt 
import seaborn as sns
import pickle 

import sklearn
from sklearn import metrics

import alibi 
from alibi_detect.cd import ChiSquareDrift, TabularDrift
from alibi_detect.saving import save_detector, load_detector

import sqlalchemy
import snowflake.connector
from sqlalchemy import create_engine
from snowflake.sqlalchemy import *

import xgboost
from datetime import datetime, timedelta
import time  
import pytz    
tz_NY = pytz.timezone('Asia/Kolkata')

import snowflake_creds
import LOS_Preprocessing

import warnings
warnings.filterwarnings('ignore')

In [11]:
# Creating the connection engine (way 1)
engine = create_engine(URL(
        account="cr21746.ap-south-1",
        user= snowflake_creds.USER_NAME,
        password= snowflake_creds.PASSWORD,
        role="ACCOUNTADMIN",
        warehouse="COMPUTE_WH",
        database="HEALTHDB",
        schema="HEALTHSCHEMA"
    ))

## Data Drift Detector Script:

In [12]:
def data_monitoring_batch_query(a):
    query = f"""

        SELECT CASE_ID,
               COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
               COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
               COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
               COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
               COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
               COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
               COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
               COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
               COALESCE(BED_GRADE,0) AS BED_GRADE,
               PATIENTID,
               COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
               COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
               COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
               COALESCE(VISITORS_WITH_PATIENT_X,0) AS VISITORS_WITH_PATIENT,
               COALESCE(AGE,'None') AS AGE,
               COALESCE(ADMISSION_DEPOSIT_X,0) AS ADMISSION_DEPOSIT,
               ADMISSION_DATE,
               DISCHARGE_DATE

            FROM HEALTHDB.HEALTHSCHEMA.TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
            WHERE ADMISSION_DATE >= CURRENT_DATE-144+{a*7} AND ADMISSION_DATE < CURRENT_DATE-144+{(a+1)*7}        

        """
    return query

In [13]:
def data_monitoring(batch_id):
    # Loading the train data
    with engine.connect() as conn:
        batch_df = pd.DataFrame(pd.read_sql(data_monitoring_batch_query(batch_id),conn))
        batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]
    
    # Getting the numerical and categorical columns for creating the datadrift object
    num_columns = ['AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL','VISITORS_WITH_PATIENT','ADMISSION_DEPOSIT']
    id_columns = ['CASE_ID','PATIENTID','ADMISSION_DATE','DISCHARGE_DATE']
    cat_columns = [col for col in batch_df.columns.tolist() if col not in num_columns+id_columns]
    
    # Getting the final prepared data
    batch_final = batch_df[num_columns + cat_columns]
    
    # Loading the Trained data drift detector
    with open('Trained_Drift_Detector.pkl','rb') as F:
        trained_drift_model = pickle.load(F)    
    
    # Checking for drift
    # If you are interested in individual feature-wise drift, this is also possible:
    fpreds = trained_drift_model.predict(batch_final.values, drift_type='feature')
    
    log_df = pd.DataFrame()
    log_df['Time Period'] = ([str(batch_df['ADMISSION_DATE'].min()) + ' to ' + 
                              str(batch_df['ADMISSION_DATE'].max())]
                              * len(batch_final.columns.tolist())
                            )
    log_df['Total Records'] = batch_df.shape[0]
    log_df['Features'] = batch_final.columns.tolist()
    log_df['Is Drift'] = fpreds['data']['is_drift']
    log_df['Stat Test'] = log_df['Features'].apply(lambda x: 'Chi2' if x in cat_columns else 'K-S')
    log_df['Stats Value'] = np.round(fpreds['data']['distance'])
    log_df['P-value'] = np.round(fpreds['data']['p_val'])
    
    return log_df

In [14]:
data_log_df = data_monitoring(2)

In [15]:
data_log_df

,Time Period,Total Records,Features,Is Drift,Stat Test,Stats Value,P-value
0,2022-12-19 to 2022-12-25,16689,AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,1,K-S,0.0,0.0
1,2022-12-19 to 2022-12-25,16689,VISITORS_WITH_PATIENT,1,K-S,0.0,0.0
2,2022-12-19 to 2022-12-25,16689,ADMISSION_DEPOSIT,1,K-S,0.0,0.0
3,2022-12-19 to 2022-12-25,16689,HOSPITAL_CODE,1,Chi2,124.0,0.0
4,2022-12-19 to 2022-12-25,16689,HOSPITAL_TYPE_CODE,1,Chi2,18.0,0.0
5,2022-12-19 to 2022-12-25,16689,CITY_CODE_HOSPITAL,1,Chi2,37.0,0.0
6,2022-12-19 to 2022-12-25,16689,HOSPITAL_REGION_CODE,1,Chi2,8.0,0.0
7,2022-12-19 to 2022-12-25,16689,DEPARTMENT,1,Chi2,21.0,0.0
8,2022-12-19 to 2022-12-25,16689,WARD_TYPE,1,Chi2,45.0,0.0
9,2022-12-19 to 2022-12-25,16689,WARD_FACILITY_CODE,1,Chi2,11.0,0.0


## Model Drift Detector Script:

In [16]:
# Function to check model drift
def check_model_drift(ref_metric_dict,cur_metric_dict,type='classification',tol=0.1):
    if type == 'classification':
        precision_change = abs((cur_metric_dict['Precision']-ref_metric_dict['Precision'])/ref_metric_dict['Precision'])
        recall_change = abs((cur_metric_dict['Recall']-ref_metric_dict['Recall'])/ref_metric_dict['Recall'])
        roc_auc_change = abs((cur_metric_dict['Roc-Auc']-ref_metric_dict['Roc-Auc'])/ref_metric_dict['Roc-Auc'])

        counter = 0
        for i in [precision_change,recall_change,roc_auc_change]:
            if i > tol:
                counter += 1

        if counter > 0:
            print("ALERT! There is a model drift.")
            print("Change in Precision: "+ str(np.round(100*precision_change,2))+"%")
            print("Change in Recall: "+ str(np.round(100*recall_change,2))+"%")
            print("Change in Roc-Auc: "+ str(np.round(100*roc_auc_change,2))+"%")
            return 1
        else:
            print("There is no model drift.")
            return 0

    elif type == 'regression':
        rmse_change = abs((cur_metric_dict['RMSE']-ref_metric_dict['RMSE'])/ref_metric_dict['RMSE'])
        mae_change = abs((cur_metric_dict['MAE']-ref_metric_dict['MAE'])/ref_metric_dict['MAE'])
        
        counter = 0
        for i in [rmse_change,mae_change]:
            if i > tol:
                counter += 1

        if counter > 0:
            print("ALERT! There is a model drift.")
            RMSE_CHANGE = np.round(100*rmse_change,2)
            MAE_CHANGE = np.round(100*mae_change,2)
            print("Change in RMSE: "+ str(np.round(100*rmse_change,2))+"%")
            print("Change in MAE: "+ str(np.round(100*mae_change,2))+"%")
            return 1, RMSE_CHANGE, MAE_CHANGE
        else:
            print("There is no model drift.")
            RMSE_CHANGE = 'NONE'
            MAE_CHANGE = 'NONE'
            return 0, RMSE_CHANGE, MAE_CHANGE
        
        

In [17]:
def model_monitoring_batch_query(a):
    query_sim = f"""

        SELECT *
        FROM TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
        WHERE ADMISSION_DATE >= CURRENT_DATE-144+{a*7} AND ADMISSION_DATE < CURRENT_DATE-144+{(a+1)*7}
        
    """
    return query_sim

In [18]:
def model_monitoring(batch_id):
    # Loading the train data
    with engine.connect() as conn:
        batch_df = pd.DataFrame(pd.read_sql(model_monitoring_batch_query(batch_id),conn))
        batch_df.columns = [col.upper() for col in batch_df.columns.tolist()]
    
#     print(batch_df.shape)
    
    # Creating the current performance dict (from scoring)
    actual = batch_df['LOS_X']
    predicted = batch_df['PREDICTED_LOS']

    rmse = np.sqrt(metrics.mean_squared_error(actual,predicted))
    mae = np.sqrt(metrics.mean_absolute_error(actual,predicted))
#     print("RMSE: ", rmse)
#     print("MAE: ", mae)

    scoring_ref_metrics = {}
    scoring_ref_metrics['RMSE'] = rmse
    scoring_ref_metrics['MAE'] = mae #+ 0.2*mae
#     print(scoring_ref_metrics)
    
    
    # Loading the reference performance dict (from training)
    with open('MODEL_XGB_PERFM_METRICS.pkl', 'rb') as F:
        model_ref_metric = pickle.load(F)
        
#     print(model_ref_metric)
    
    # Check for model drift
    model_drift, RMSE_CHANGE, MAE_CHANGE = check_model_drift(model_ref_metric,scoring_ref_metrics,type='regression',tol=0.1)
    
    # Log values
    log = {}
    log['Time Period'] = str(batch_df['ADMISSION_DATE'].min()) + ' to ' + str(batch_df['ADMISSION_DATE'].max())
    log['Total Records'] = batch_df.shape[0]
    log['Scoring Metrics'] = scoring_ref_metrics
    log['Training Metrics'] = model_ref_metric
    log['Model Drift IND'] = model_drift
    log['RMSE Change'] = RMSE_CHANGE
    log['MAE Change'] = MAE_CHANGE
    
    return log
    

In [19]:
model_log_dict = model_monitoring(2)

There is no model drift.


In [20]:
model_log_dict

{'Time Period': '2022-12-19 to 2022-12-25',
 'Total Records': 16689,
 'Scoring Metrics': {'RMSE': 14.879287386202929, 'MAE': 3.298999279937221},
 'Training Metrics': {'RMSE': 13.607345304032433, 'MAE': 3.1702232476032353},
 'Model Drift IND': 0,
 'RMSE Change': 'NONE',
 'MAE Change': 'NONE'}

# Retraining Helper Functions:

In [21]:
def check_n_create_model_features(df,feat_list):
    test = pd.DataFrame()
    for col in feat_list:
        if col in df.columns.tolist():
            test[col] = df[col]
        else:
            test[col] = 0
    
    return test

def feature_selection(df):
    # Creating X and Y
    x_train = df.drop('LOS',axis=1)
    y_train = df[['LOS']]
    
    # Decision Tree
    from sklearn.tree import DecisionTreeRegressor

    dtree = DecisionTreeRegressor()
    dtree.fit(x_train,y_train)
    # Feature Importance
    feat_imp = (pd.DataFrame(zip(x_train.columns,dtree.feature_importances_),columns=['feature','imp'])
                .sort_values(by='imp',ascending=False))
    final_features_dtree = feat_imp[feat_imp['imp']>=0.01]['feature'].values.tolist()
    
    # XGBoost
    import xgboost as xgb

    xgb_ = xgb.XGBRegressor()
    xgb_.fit(x_train,y_train)
    # Feature Importance
    feat_imp = (pd.DataFrame(zip(x_train.columns,xgb_.feature_importances_),columns=['feature','imp'])
                .sort_values(by='imp',ascending=False))
    final_features_xgb = feat_imp[feat_imp['imp']>=0.01]['feature'].values.tolist()
    
    model_features =  list(set(final_features_dtree).union(set(final_features_xgb)))
    print("Final Features from both Dtree & XGB: "+str(len(model_features)))
    
    import pickle

    final_feats_list = model_features+['LOS']

    with open('./Retraining Artifacts/MODEL_FEATS.pkl','wb') as F:
        pickle.dump(final_feats_list,F)
    
    return final_feats_list
    

def retraining_batch_query(max_date):
    query = f"""

        WITH TRAIN_BASE AS (

            SELECT CASE_ID,
                   COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                   COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                   COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                   COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                   COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                   COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                   COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                   COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                   COALESCE(BED_GRADE,0) AS BED_GRADE,
                   PATIENTID,
                   COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                   COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                   COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                   COALESCE(VISITORS_WITH_PATIENT,0) AS VISITORS_WITH_PATIENT,
                   COALESCE(AGE,'None') AS AGE,
                   COALESCE(ADMISSION_DEPOSIT,0) AS ADMISSION_DEPOSIT,
                   ADMISSION_DATE,
                   DISCHARGE_DATE

            FROM HEALTHDB.HEALTHSCHEMA.HEALTH_DATA
            WHERE ADMISSION_DATE >= '2022-11-01' --- To reduce the load

        ),

        TRAIN_BASE_WITH_FEATURES AS (

            SELECT *,
                    MONTHNAME(ADMISSION_DATE) AS ADMISSION_MONTH,
                    DAYNAME(ADMISSION_DATE) AS ADMISSION_DAY,    
                    CONCAT(TYPE_OF_ADMISSION,'-',SEVERITY_OF_ILLNESS) AS ADMISSION_ILLNESS,
                    CONCAT(SEVERITY_OF_ILLNESS,'-',BED_GRADE) AS ILLNESS_BEDGRADE,
                    CONCAT(DEPARTMENT,'-',SEVERITY_OF_ILLNESS) AS DEPARTMENT_ILLNESS,
                    DATEDIFF(day,ADMISSION_DATE,DISCHARGE_DATE) AS LOS
            FROM TRAIN_BASE 

        ),    

        NEW_DATA_WITH_FEATURES AS (

             SELECT CASE_ID,
                       COALESCE(HOSPITAL_CODE,0) AS HOSPITAL_CODE,
                       COALESCE(HOSPITAL_TYPE_CODE,'None') AS HOSPITAL_TYPE_CODE,
                       COALESCE(CITY_CODE_HOSPITAL,0) AS CITY_CODE_HOSPITAL,
                       COALESCE(HOSPITAL_REGION_CODE,'None') AS HOSPITAL_REGION_CODE,
                       COALESCE(AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL_X,0) AS AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL,
                       COALESCE(DEPARTMENT,'None') AS DEPARTMENT,
                       COALESCE(WARD_TYPE,'None') AS WARD_TYPE,
                       COALESCE(WARD_FACILITY_CODE,'None') AS WARD_FACILITY_CODE,
                       COALESCE(BED_GRADE,0) AS BED_GRADE,
                       PATIENTID,
                       COALESCE(CITY_CODE_PATIENT,0) AS CITY_CODE_PATIENT,
                       COALESCE(TYPE_OF_ADMISSION,'None') AS TYPE_OF_ADMISSION,
                       COALESCE(SEVERITY_OF_ILLNESS,'Minor') AS SEVERITY_OF_ILLNESS,
                       COALESCE(VISITORS_WITH_PATIENT_X,0) AS VISITORS_WITH_PATIENT,
                       COALESCE(AGE,'None') AS AGE,
                       COALESCE(ADMISSION_DEPOSIT_X,0) AS ADMISSION_DEPOSIT,
                       ADMISSION_DATE,
                       DISCHARGE_DATE,
                       ADMISSION_MONTH,
                       ADMISSION_DAY,
                       ADMISSION_ILLNESS,
                       ILLNESS_BEDGRADE,
                       DEPARTMENT_ILLNESS,
                       LOS_X AS LOS 
                    FROM HEALTHDB.HEALTHSCHEMA.TEMP_LOS_PREDICTION_MODEL_LOGGING_TABLE_HARI
                    WHERE ADMISSION_DATE < '{max_date}'

        )


        SELECT * FROM TRAIN_BASE_WITH_FEATURES
        UNION ALL
        SELECT * FROM NEW_DATA_WITH_FEATURES;

        """
    return query

def retrain_model(cut_off_date):
    with engine.connect() as conn:
    
        # Loading the scoring data
        data = pd.DataFrame(pd.read_sql(retraining_batch_query(cut_off_date),conn))
        data.columns = [col.upper() for col in data.columns.tolist()]
        print(data.shape)
    #     display(data.head())

        # Splitting the data into Train and Test set
        import pytz    
        from datetime import datetime, timedelta
        tz_NY = pytz.timezone('Asia/Kolkata')

        max_date = data.ADMISSION_DATE.max()
        min_date = max_date - timedelta(days=7)

        data_train = data[(data['ADMISSION_DATE'] <= min_date)]
        data_test = data[(data['ADMISSION_DATE'] >= min_date) & (data['ADMISSION_DATE'] <= max_date)]


        # Applying the preprocessing steps
        df_train_processed = LOS_Preprocessing.preprocess_data(data_train)
        print(df_train_processed.shape)

        df_test_processed = LOS_Preprocessing.preprocess_data(data_test)
        print(df_test_processed.shape)

        # Performing feature selection
        df_final = df_train_processed.copy()
        print(df_final.shape)
    #     display(df_final.head())
        print("Feature Selection Started..")
        model_feats = feature_selection(df_final)
        print(model_feats)
        model_feats.remove('LOS')

        # Model Building
        import xgboost as xgb

        xgb_ = xgb.XGBRegressor()
        xgb_.fit(df_final[model_feats],df_final['LOS'])

        df_test_final = check_n_create_model_features(df_test_processed,model_feats)
        if 'LOS' in df_test_final.columns.tolist():
            df_test_final = df_test_final.drop('LOS',axis=1)
        preds = np.ceil(xgb_.predict(df_test_final))
        rmse = np.sqrt(metrics.mean_squared_error(df_test_processed['LOS'],preds))
        mae = np.sqrt(metrics.mean_absolute_error(df_test_processed['LOS'],preds))
        print("\n Test Performance (new model)")
        print("RMSE: ", rmse)
        print("MAE: ", mae)      

        # Saving the trained model
        booster = xgb_.get_booster()
        booster.save_model('./Retraining Artifacts/MODEL_XGB.model')

        model_xgb_metrics_new = {}
        model_xgb_metrics_new['RMSE'] = rmse
        model_xgb_metrics_new['MAE'] = mae

        import pickle

        with open('./Retraining Artifacts/MODEL_XGB_PERFM_METRICS.pkl','wb') as F:
            pickle.dump(model_xgb_metrics_new,F)


        # Getting the predictions from the old model
        model = xgboost.XGBRegressor()
        model.load_model('MODEL_XGB.model')
    #     df_test_processed['PREDICTED_LOS'] = np.ceil(model.predict(df_test_processed[model_feats]))

        with open('MODEL_FEATS.pkl','rb') as F:
            model_feats_old = pickle.load(F)

        df_test_final = check_n_create_model_features(df_test_processed,model_feats_old)
        if 'LOS' in df_test_final.columns.tolist():
            df_test_final = df_test_final.drop('LOS',axis=1)
        preds = np.ceil(model.predict(df_test_final))
        rmse = np.sqrt(metrics.mean_squared_error(df_test_processed['LOS'],preds))
        mae = np.sqrt(metrics.mean_absolute_error(df_test_processed['LOS'],preds))
        print("\n Test Performance (old model)")
        print("RMSE: ", rmse)
        print("MAE: ", mae)   

        model_xgb_metrics_old = {}
        model_xgb_metrics_old['RMSE'] = rmse
        model_xgb_metrics_old['MAE'] = mae
    
    return model_xgb_metrics_new, model_xgb_metrics_old


def finalize_model(new_perform_dict, old_perform_dict):
    count = 0
    for metric in new_perform_dict.keys():
        if new_perform_dict[metric] < old_perform_dict[metric]:
            count += 1
    
    if count > 0:
        return 'New Model'
    else:
        return 'Old Model'

    
def deploy_model(selector='Old Model'):
    if selector != 'Old Model':
        # STEP-1:
        # Loading the old model
        with open('MODEL_FEATS.pkl','rb') as F:
            old_feats = pickle.load(F)
        with open('MODEL_XGB.model','rb') as F:
            old_model = pickle.load(F)
        with open('MODEL_XGB_PERFM_METRICS.pkl','rb') as F:
            old_perfm_dict = pickle.load(F)
        
        # Saving the copy to Archive folder
        with open('./Archive/MODEL_FEATS.pkl','wb') as F:
            pickle.dump(old_feats,F)
        with open('./Archive/MODEL_XGB.model','wb') as F:
            pickle.dump(old_model,F)
        with open('./Archive/MODEL_XGB_PERFM_METRICS.pkl','wb') as F:
            pickle.dump(old_perfm_dict,F)
        
        # STEP-2:
        # Loadin the new model
        with open('./Retraining Artifacts/MODEL_FEATS.pkl','rb') as F:
            new_feats = pickle.load(F)
        with open('./Retraining Artifacts/MODEL_XGB.model','rb') as F:
            new_model = pickle.load(F)
        with open('./Retraining Artifacts/MODEL_XGB_PERFM_METRICS.pkl','rb') as F:
            new_perfm_dict = pickle.load(F)
        
        # Replacing the old model artifacts with the new model
        with open('MODEL_FEATS.pkl','wb') as F:
            pickle.dump(new_feats,F)
        with open('MODEL_XGB.model','wb') as F:
            pickle.dump(new_model,F)
        with open('MODEL_XGB_PERFM_METRICS.pkl','wb') as F:
            pickle.dump(new_perfm_dict,F)
        
    return 'Deployment Successful'


# Retraining Trigger:

In [22]:
# Data drift condition
data_drift = data_log_df['Is Drift'].sum() > 0
data_drift

True

In [24]:
# Model drift condition
labels = [False, True]
model_drift = labels[model_log_dict['Model Drift IND']]
model_drift

False

In [25]:
# Max date for retraining  (cut-off date)
cut_off_date = model_log_dict['Time Period'].split(' ')[2]
cut_off_date

'2022-12-25'

In [28]:
# Retraining trigger condition
if data_drift == True or model_drift == True:
    # Do retraining
    print("Retraining Started...")
    new_dict, old_dict = retrain_model(cut_off_date)
    
    # Finalize the model
    print("\nModel Selection Started...")
    select_model = finalize_model(new_perform_dict=new_dict, old_perform_dict=old_dict)
    
    # Deploy the selected model
    print("\nDeployment Started...")
    deploy_model(select_model)
    

Retraining Started...
(104034, 25)
(87302, 147)
(19020, 144)
(87302, 147)
Feature Selection Started..
Final Features from both Dtree & XGB: 25
['ADMISSION_DAY_Thu', 'CITY_CODE_HOSPITAL_7', 'ADMISSION_DEPOSIT', 'VISITORS_WITH_PATIENT', 'SEVERITY_OF_ILLNESS_Minor', 'AGE_71-80', 'CITY_CODE_PATIENT_8', 'TYPE_OF_ADMISSION_Trauma', 'ADMISSION_DAY_Fri', 'ILLNESS_BEDGRADE_Extreme-1', 'CITY_CODE_HOSPITAL_2', 'ADMISSION_MONTH_Nov', 'AVAILABLE_EXTRA_ROOMS_IN_HOSPITAL', 'ADMISSION_DAY_Tue', 'AGE_31-40', 'ILLNESS_BEDGRADE_Moderate-2', 'ADMISSION_MONTH_Dec', 'ADMISSION_DAY_Sun', 'ADMISSION_DAY_Sat', 'AGE_41-50', 'TYPE_OF_ADMISSION_Emergency', 'WARD_TYPE_S', 'WARD_TYPE_P', 'WARD_TYPE_Q', 'BED_GRADE_2', 'LOS']

 Test Performance (new model)
RMSE:  15.04027890315711
MAE:  3.3200471282429023

 Test Performance (old model)
RMSE:  14.866728319837454
MAE:  3.3015705404623406

Model Selection Started...

Deployment Started...
